### Import libraries

In [40]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.optimizers import SGD
import sklearn as sk
import sklearn.model_selection

### Extract labels for images

In [41]:
result_frame = pd.read_csv('dataset/subsystem_1/Dataset_Subsystem_1.csv')

In [42]:
result_frame

,ID,source,frame,camera_facing_side,gesture,palm_root_x,palm_root_y,palm_thumb_1_x,palm_thumb_1_y,palm_thumb_2_x,...,dorsal_ring_4_x,dorsal_ring_4_y,dorsal_pinky_1_x,dorsal_pinky_1_y,dorsal_pinky_2_x,dorsal_pinky_2_y,dorsal_pinky_3_x,dorsal_pinky_3_y,dorsal_pinky_4_x,dorsal_pinky_4_y
0,102,open_palm.webm,0,open,palm,279,369,189,332,137,...,0,0,0,0,0,0,0,0,0,0
1,102,open_palm.webm,1,open,palm,279,370,188,331,137,...,0,0,0,0,0,0,0,0,0,0
2,102,open_palm.webm,2,open,palm,279,370,187,331,137,...,0,0,0,0,0,0,0,0,0,0
3,102,open_palm.webm,3,open,palm,278,370,186,330,136,...,0,0,0,0,0,0,0,0,0,0
4,102,open_palm.webm,4,open,palm,278,371,185,329,136,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11991,723,three_fingers_dorsal.webm,197,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,312,0,0,0,0,0,0
11992,723,three_fingers_dorsal.webm,198,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0
11993,723,three_fingers_dorsal.webm,199,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0
11994,723,three_fingers_dorsal.webm,200,three_fingers,dorsal,0,0,0,0,0,...,0,0,800,311,0,0,0,0,0,0


In [43]:
result_frame_102_159 = result_frame[(result_frame['ID'] >= 102) & (result_frame['ID'] <= 300)  & (result_frame['frame'] < 10)]

In [44]:
Y = result_frame_102_159.values[:,5:]

In [45]:
mu = np.mean(Y[:,i])
sigma = np.std(Y[:,i])

for i in range(Y.shape[1]):
    Y[:,i] = (Y[:,i] - mu)/sigma

In [46]:
Y

array([[1.0240334168071514, 1.6025430189928067, 0.44552381462149604, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838],
       [1.0240334168071514, 1.6089709034615363, 0.43909593015276654, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838],
       [1.0240334168071514, 1.6089709034615363, 0.43266804568403705, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838],
       ...,
       [-0.76934634996838, -0.76934634996838, -0.76934634996838, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838],
       [-0.76934634996838, -0.76934634996838, -0.76934634996838, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838],
       [-0.76934634996838, -0.76934634996838, -0.76934634996838, ...,
        -0.76934634996838, -0.76934634996838, -0.76934634996838]],
      dtype=object)

### Extract frames from videos and transform them into images (do not run: too much data for the RAM)

In [47]:
train_folders = ['102','159', '294'] #, '441', '564', '576', '609','666','711','723'
images_array = []
gestures = ['open_palm','open_dorsal','fist_palm','fist_dorsal','three_fingers_palm','three_fingers_dorsal']
for folder in train_folders:
    for gesture in gestures:
        i = 0
        video_reader = cv2.VideoCapture("dataset/subsystem_1/videos/"+folder+"/"+gesture+".webm")
        ret, frame = video_reader.read()
        while ret and i < 10:
            ret, frame = video_reader.read()
            i += 1
            if not ret:
                continue
            gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            images_array.append(gray_frame)
X = np.array(images_array)
X.shape

(180, 480, 640)

In [48]:
X = X.reshape((-1,480,640,1))

In [49]:
X_train,X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)

In [50]:
X_train.shape

(144, 480, 640, 1)

### Convert data to be usable by Tensorflow and Keras

In [51]:
X_train = tf.convert_to_tensor(X_train,np.float32)
X_test = tf.convert_to_tensor(X_test,np.float32)
Y_train = tf.convert_to_tensor(Y_train,np.float32)
Y_test = tf.convert_to_tensor(Y_test,np.float32)
# dataset = tf.data.Dataset.from_tensor_slices((X, Y))
# batched_dataset = dataset.batch(4)

### Create model to extract landmarks

In [52]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(480,640,1)),
    tf.keras.layers.Conv2D(1,3,input_shape = (480,640), activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(500,activation='relu'),
    tf.keras.layers.Dense(80)])

sgd = SGD(learning_rate=.001)
model.compile(optimizer=sgd,
              loss='mean_squared_error')

### Train the model

In [54]:
model.fit(X_train,Y_train,epochs=100,batch_size=4)

Train on 144 samples
Epoch 1/100
144/144 [==============================] - 15s 105ms/sample - loss: 0.7386
Epoch 2/100
144/144 [==============================] - 16s 110ms/sample - loss: 0.7008
Epoch 3/100
144/144 [==============================] - 15s 106ms/sample - loss: 0.6451
Epoch 4/100
144/144 [==============================] - 16s 109ms/sample - loss: 0.5899
Epoch 5/100
144/144 [==============================] - 15s 107ms/sample - loss: 0.5184
Epoch 6/100
144/144 [==============================] - 16s 111ms/sample - loss: 0.4830
Epoch 7/100
144/144 [==============================] - 17s 115ms/sample - loss: 0.4575
Epoch 8/100
144/144 [==============================] - 15s 103ms/sample - loss: 0.3666
Epoch 9/100
144/144 [==============================] - 16s 114ms/sample - loss: 0.3384
Epoch 10/100
144/144 [==============================] - 17s 115ms/sample - loss: 0.3121
Epoch 11/100
144/144 [==============================] - 15s 101ms/sample - loss: 0.2823
Epoch 12/100
144/144

144/144 [==============================] - 12s 83ms/sample - loss: 0.0057
Epoch 95/100
144/144 [==============================] - 12s 83ms/sample - loss: 0.0059
Epoch 96/100
144/144 [==============================] - 13s 91ms/sample - loss: 0.0065
Epoch 97/100
144/144 [==============================] - 12s 85ms/sample - loss: 0.0068
Epoch 98/100
144/144 [==============================] - 12s 85ms/sample - loss: 0.0066
Epoch 99/100
144/144 [==============================] - 12s 83ms/sample - loss: 0.0067
Epoch 100/100
144/144 [==============================] - 12s 85ms/sample - loss: 0.0058


In [37]:
Y_pred = model.predict(np.array([X_test[-5:-3]]))

SystemError: <class 'tensorflow.python.framework.ops.EagerTensor'> returned a result with an error set

In [29]:
Y_pred.shape

(5, 80)

In [30]:
Y_pred

array([[-1.1663228 , -1.238128  , -1.1690909 , -0.66732115, -0.36737728,
        -0.56023735, -0.7314189 , -1.0209523 , -0.24925569,  0.07386892,
        -0.97039986, -0.8394834 , -1.3401117 , -0.9059081 , -0.8636174 ,
        -1.2789137 , -0.10039742, -0.64424765, -0.32916346, -0.3484783 ,
        -0.8504143 , -1.1269178 , -0.6096702 , -0.8476262 , -1.2003499 ,
        -0.75489795, -0.6353779 , -0.2103592 , -0.16079943, -0.18568227,
        -0.23860274, -0.07216985, -0.5282693 , -0.7234217 , -0.4372708 ,
        -0.30305007, -0.74704325, -0.8163071 , -0.11149857, -0.6217943 ,
         0.3666081 ,  0.5442127 ,  0.19574606,  0.1461442 , -1.0188718 ,
        -1.1370198 , -0.95535475, -0.41047585,  0.94425845,  0.32350302,
        -1.1047246 , -1.09559   , -0.6962515 , -1.0528497 , -0.50909734,
        -1.0312866 ,  0.8313675 , -0.34283063, -0.8316379 , -1.2918135 ,
        -1.0853971 , -0.8961147 , -1.092424  , -1.0281684 ,  1.4366953 ,
        -0.17025796, -1.9411149 , -1.3121059 , -0.8

In [32]:
Y_test[-5]

<tf.Tensor: shape=(80,), dtype=float32, numpy=
array([-0.9513901 , -0.9827279 , -0.95261145, -0.970333  , -0.842435  ,
       -0.8591855 , -0.8194345 , -0.85605085, -0.6792668 , -0.6899612 ,
       -0.67520887, -0.6768204 , -0.6723134 , -0.6643044 , -0.66847056,
       -0.64179105, -0.6750829 , -0.68854415, -0.67333466, -0.67596203,
       -0.6702064 , -0.65710056, -0.6671764 , -0.63418496, -0.43874317,
       -0.4386913 , -0.43961322, -0.43380353, -0.4398355 , -0.4235086 ,
       -0.43994004, -0.41345853, -0.4407254 , -0.4410239 , -0.4415228 ,
       -0.43731782, -0.44221422, -0.433162  , -0.442963  , -0.4274365 ,
        1.29945   ,  1.3170754 ,  1.6920677 ,  1.6841321 ,  1.7746836 ,
        1.7109845 ,  1.8359953 ,  1.6975476 ,  1.1606627 ,  1.1562359 ,
        0.9700852 ,  1.1935923 ,  0.9835692 ,  0.9383982 ,  1.2309161 ,
        1.0875882 ,  0.96241766,  1.3191495 ,  0.71249545,  1.3308587 ,
        0.5920473 ,  1.1390995 ,  0.7077506 ,  1.2882547 ,  0.85606945,
        1.4394726

In [ ]:
print(X.shape)
print(Y.shape)

In [ ]:
model.fit(batched_dataset)

In [ ]:
print(Y)

In [ ]:
model.predict(np.array([X[0]]))

In [ ]:
sk.model_selection.cross_val_score(model, X, Y, cv=5, scoring='neg_mean_squared_error')

### Validate the model

In [ ]:
model.metrics_squared_error(X_test,Y_test)